In [1]:
import re
from collections import OrderedDict

def read_srt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
        blocks = re.split(r'\n\n', content)
        subtitles = OrderedDict()

        for block in blocks:
            if block.strip() == "":
                continue

            lines = block.split('\n')
            if len(lines) >= 3:
                # Extract timestamp
                timestamp = lines[1].strip()

                # Combine subtitle lines
                subtitle = " ".join(lines[2:]).strip()

                # Add subtitle and timestamp to ordered dictionary
                subtitles[subtitle] = timestamp

    return subtitles

# Usage
srt_file_path = "Cost To Start.srt"
subtitles_dict = read_srt(srt_file_path)

for subtitle, timestamp in subtitles_dict.items():
    print(f"{subtitle}: {timestamp}")

So how much does it take to get started on Airbnb?: 00:00:03,668 --> 00:00:06,551
Well, it depends.: 00:00:06,971 --> 00:00:07,612
If you're going to be renting a new space,: 00:00:07,812 --> 00:00:09,614
it's going to cost you different than if you already: 00:00:09,754 --> 00:00:11,296
have your home.: 00:00:11,356 --> 00:00:11,936
Your most likely already paying rent or paying a mortgage: 00:00:12,477 --> 00:00:14,839
if you already have a space.: 00:00:14,919 --> 00:00:15,960
So factor that into your monthly cost.: 00:00:16,341 --> 00:00:18,843
But let's talk about how much extra you'll need: 00:00:19,083 --> 00:00:21,005
to put into the space per room.: 00:00:21,025 --> 00:00:22,987
Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,: 00:00:23,530 --> 00:00:29,932
fixtures, everything that people will need like plates and sheets and towels for a one: 00:00:30,212 --> 00:00:34,254
bedroom, and then about $2,000 per bedroom and bathroom after that

In [2]:
subtitles_dict

OrderedDict([('So how much does it take to get started on Airbnb?',
              '00:00:03,668 --> 00:00:06,551'),
             ('Well, it depends.', '00:00:06,971 --> 00:00:07,612'),
             ("If you're going to be renting a new space,",
              '00:00:07,812 --> 00:00:09,614'),
             ("it's going to cost you different than if you already",
              '00:00:09,754 --> 00:00:11,296'),
             ('have your home.', '00:00:11,356 --> 00:00:11,936'),
             ('Your most likely already paying rent or paying a mortgage',
              '00:00:12,477 --> 00:00:14,839'),
             ('if you already have a space.', '00:00:14,919 --> 00:00:15,960'),
             ('So factor that into your monthly cost.',
              '00:00:16,341 --> 00:00:18,843'),
             ("But let's talk about how much extra you'll need",
              '00:00:19,083 --> 00:00:21,005'),
             ('to put into the space per room.',
              '00:00:21,025 --> 00:00:22,987'),
     

In [3]:

!pip install tiktoken

In [4]:
import tiktoken

def count_tokens(text):
    """Count the number of tokens in a given text with tiktoken"""
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokenized_text = tokenizer.encode(text)

    return len(tokenized_text)

In [18]:
def build_subtitle_groups(subtitles_dict, max_tokens=2000):
    groups = []
    current_group = []
    current_token_count = 0

    for subtitle, timestamp in subtitles_dict.items():
        subtitle_with_newline = subtitle + "\n"
        subtitle_token_count = count_tokens(subtitle_with_newline)

        if current_token_count + subtitle_token_count <= max_tokens:
            current_group.append(subtitle_with_newline)
            current_token_count += subtitle_token_count
        else:
            groups.append(current_group)
            current_group = [subtitle_with_newline]
            current_token_count = subtitle_token_count

    if current_group:
        groups.append(current_group)

    return groups

# Usage
subtitle_groups = build_subtitle_groups(subtitles_dict)

for i, group in enumerate(subtitle_groups):
    print(f"Group {i + 1}:")
    for subtitle in group:
        print(subtitle, end="")
    print("\n---\n")


Group 1:
So how much does it take to get started on Airbnb?
Well, it depends.
If you're going to be renting a new space,
it's going to cost you different than if you already
have your home.
Your most likely already paying rent or paying a mortgage
if you already have a space.
So factor that into your monthly cost.
But let's talk about how much extra you'll need
to put into the space per room.
Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,
fixtures, everything that people will need like plates and sheets and towels for a one
bedroom, and then about $2,000 per bedroom and bathroom after that.
And the reason why is it's not going to quite cost you $2,000 per bedroom and bathroom,
but as you gain larger and square feet, you're going to need to fill the space up.
Now, I'm not going to tell you to go thrifting for...
cheaper furniture, which you can do. If you've got a really good eye for creating unique and
lovely spaces by thrifting, you can do this

In [20]:
subtitle_groups[0]

['So how much does it take to get started on Airbnb?\n',
 'Well, it depends.\n',
 "If you're going to be renting a new space,\n",
 "it's going to cost you different than if you already\n",
 'have your home.\n',
 'Your most likely already paying rent or paying a mortgage\n',
 'if you already have a space.\n',
 'So factor that into your monthly cost.\n',
 "But let's talk about how much extra you'll need\n",
 'to put into the space per room.\n',
 'Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,\n',
 'fixtures, everything that people will need like plates and sheets and towels for a one\n',
 'bedroom, and then about $2,000 per bedroom and bathroom after that.\n',
 "And the reason why is it's not going to quite cost you $2,000 per bedroom and bathroom,\n",
 "but as you gain larger and square feet, you're going to need to fill the space up.\n",
 "Now, I'm not going to tell you to go thrifting for...\n",
 "cheaper furniture, which you can do. If you've 

In [116]:
def get_subtitle_summary(subtitles_str):
    """
    This function returns the summarized subtitles using ChatGPT
    """
    print("Getting summary from ChatGPT for the subtitles")
    raw_subtitles = repr(subtitles_str)
    prompt = f"Return the exact same text, but remove the sentences that are not essential (sentences are separated by new lines '\\n') to the message in order to shorten the text. Keep full sentences exactly as the original text (do not cut them short before the '\\n'). Separate output with new lines:\n{raw_subtitles}"
    print(prompt)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            # {"role": "system", "content": "You are an helpfull AI assistant"},
            {"role": "user", "content": prompt},
        ]
    )
    return completion.choices[0]['message']['content']


In [117]:
!pip install openai

In [1]:
OPENAI_KEY = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [120]:
import openai

openai.api_key = OPENAI_KEY

subtitle_group_strings = [''.join(group) for group in subtitle_groups]

summaries = []
for i, group_str in enumerate(subtitle_group_strings):
    summary = get_subtitle_summary(group_str)
    summaries.append(summary)
    print(f"Summary for group {i + 1}:\n{summary}\n---\n")



Getting summary from ChatGPT for the subtitles
Return the exact same text, but remove the sentences that are not essential (sentences are separated by new lines '\n') to the message in order to shorten the text. Keep full sentences exactly as the original text (do not cut them short before the '\n'). Separate output with new lines:
"So how much does it take to get started on Airbnb?\nWell, it depends.\nIf you're going to be renting a new space,\nit's going to cost you different than if you already\nhave your home.\nYour most likely already paying rent or paying a mortgage\nif you already have a space.\nSo factor that into your monthly cost.\nBut let's talk about how much extra you'll need\nto put into the space per room.\nNow, I have a typical rule set here that I expected to cost about $4,000 for furniture,\nfixtures, everything that people will need like plates and sheets and towels for a one\nbedroom, and then about $2,000 per bedroom and bathroom after that.\nAnd the reason why is 

In [121]:
print(summaries[0])

"So how much does it take to get started on Airbnb?
Well, it depends.
If you're going to be renting a new space,
it's going to cost you different than if you already
have your home.
Your most likely already paying rent or paying a mortgage
if you already have a space.
But let's talk about how much extra you'll need
to put into the space per room.
Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,
fixtures, everything that people will need like plates and sheets and towels for a one
bedroom, and then about $2,000 per bedroom and bathroom after that.
And the reason why is it's not going to quite cost you $2,000 per bedroom and bathroom,
but as you gain larger and square feet, you're going to need to fill the space up.
The moral of the story here is as you grow, you're going to need to know what it costs outside of your business.
So plan for $4,000 for a one-bedroom and $2,000 per room after that, and if it really matters to save that money,
then you c

In [122]:
list_of_subtitles = summaries[0][1:-1].split("\n")
list_of_subtitles

['So how much does it take to get started on Airbnb?',
 'Well, it depends.',
 "If you're going to be renting a new space,",
 "it's going to cost you different than if you already",
 'have your home.',
 'Your most likely already paying rent or paying a mortgage',
 'if you already have a space.',
 "But let's talk about how much extra you'll need",
 'to put into the space per room.',
 'Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,',
 'fixtures, everything that people will need like plates and sheets and towels for a one',
 'bedroom, and then about $2,000 per bedroom and bathroom after that.',
 "And the reason why is it's not going to quite cost you $2,000 per bedroom and bathroom,",
 "but as you gain larger and square feet, you're going to need to fill the space up.",
 "The moral of the story here is as you grow, you're going to need to know what it costs outside of your business.",
 'So plan for $4,000 for a one-bedroom and $2,000 per room after 

In [123]:
def create_filtered_dict(summaries, subtitles_dict):
    filtered_dict = OrderedDict()

    for summary in summaries:
        list_of_subtitles = summary[1:-1].split("\n")
        for subtitle in list_of_subtitles:
            if subtitle in subtitles_dict.keys():
                filtered_dict[subtitle] = subtitles_dict[subtitle]

    return filtered_dict

In [124]:
new_dict = create_filtered_dict(summaries, subtitles_dict)

In [125]:
print(new_dict)

OrderedDict([('So how much does it take to get started on Airbnb?', '00:00:03,668 --> 00:00:06,551'), ('Well, it depends.', '00:00:06,971 --> 00:00:07,612'), ("If you're going to be renting a new space,", '00:00:07,812 --> 00:00:09,614'), ("it's going to cost you different than if you already", '00:00:09,754 --> 00:00:11,296'), ('have your home.', '00:00:11,356 --> 00:00:11,936'), ('Your most likely already paying rent or paying a mortgage', '00:00:12,477 --> 00:00:14,839'), ('if you already have a space.', '00:00:14,919 --> 00:00:15,960'), ("But let's talk about how much extra you'll need", '00:00:19,083 --> 00:00:21,005'), ('to put into the space per room.', '00:00:21,025 --> 00:00:22,987'), ('Now, I have a typical rule set here that I expected to cost about $4,000 for furniture,', '00:00:23,530 --> 00:00:29,932'), ('fixtures, everything that people will need like plates and sheets and towels for a one', '00:00:30,212 --> 00:00:34,254'), ('bedroom, and then about $2,000 per bedroom a

In [126]:
import os
import subprocess
import shutil
from pathlib import Path

In [147]:
def split_video(input_video, filtered_subtitles_dict, output_video):
    # Create a temporary folder for storing video chunks
    temp_folder = "temp"
    os.makedirs(temp_folder, exist_ok=True)

    # Split video into chunks using the timestamps from the filtered dictionary
    chunks = []
    timestamps = list(filtered_subtitles_dict.values())

    for i, timestamp in enumerate(timestamps):
        start_time = timestamp.split(" --> ")[0].replace(",", ".")
        end_time = timestamp.split(" --> ")[1].replace(",", ".")
        chunk_name = f"{temp_folder}/chunk_{i:04d}.mp4"
        chunks.append(chunk_name)

        # Split the video using FFmpeg
        split_command = f'ffmpeg -y -i "{input_video}" -ss {start_time} -to {end_time} -c:v libx264 -crf 23 -preset medium -c:a aac -b:a 128k -reset_timestamps 1 "{chunk_name}"'
        subprocess.run(split_command, shell=True, check=True)

    # Create a text file containing the paths to the video chunks
    concat_list_path = f"{temp_folder}/concat_list.txt"
    with open(concat_list_path, "w") as concat_list_file:
        concat_list_file.writelines(f"file '{os.path.basename(chunk)}'\n" for chunk in chunks)

    # Join the chunks into a single video using the FFmpeg concat demuxer
    # join_command = f'ffmpeg -y -f concat -safe 0 -i "{concat_list_path}" -c copy "{output_video}"'
    join_command = f'ffmpeg -y -f concat -safe 0 -i "{concat_list_path}" -c copy "{output_video}"'
    subprocess.run(join_command, shell=True, check=True)

    # Clean up the temporary folder
    shutil.rmtree(temp_folder)

In [148]:
input_video = "Cost To Start.mp4"
output_video = "Cost To Start - filtered.mp4"
split_video(input_video, new_dict, output_video)

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1671040356584/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1671040356584/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --ena

In [149]:
def generate_srt(video_path, output_dir='temp'):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Extract audio from the input video
    audio_path = os.path.join(output_dir, 'temp_audio.wav')
    extract_audio_command = f'ffmpeg -y -i "{video_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_path}"'
    subprocess.run(extract_audio_command, shell=True, check=True)

    # Generate SRT using whisperx
    srt_path = os.path.join(output_dir, 'temp_srt.srt')
    whisperx_command = f'whisperx --model small --output_dir "{output_dir}" --output_format srt "{audio_path}"'
    subprocess.run(whisperx_command, shell=True, check=True)

    return srt_path

In [150]:
generate_srt(input_video)

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1671040356584/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1671040356584/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --ena

>>Performing VAD...
>>Performing transcription...
~~ Transcribing VAD chunk: (00:03.586 --> 00:23.330) ~~
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:03.360]  So how much does it take to get started on Airbnb?
[00:03.360 --> 00:04.200]  Well, it depends.
[00:04.200 --> 00:06.160]  If you're going to be renting a new space,
[00:06.160 --> 00:07.200]  it's gonna cost you different
[00:07.200 --> 00:08.840]  than if you already have your home.
[00:08.840 --> 00:10.520]  You're most likely already paying rent
[00:10.520 --> 00:12.720]  or paying a mortgage if you already have a space.
[00:12.720 --> 00:15.480]  So factor that into your monthly costs,
[00:15.480 --> 00:17.040]  but let's talk about how much extra
[00:17.040 --> 00:19.640]  you'll need to put into the space per room.
~~ Transcribing VAD chunk: (00:23.330 --> 00:49.621) ~~
Detecting language using up to the first 30 seconds. Use `--

'temp/temp_srt.srt'